In [2]:
import pandas as pd
import numpy as np

In [7]:
# Loading all data bases 

branchements = pd.read_csv('../data_SUEZ/branchements.csv')
canalisations = pd.read_csv('../data_SUEZ/canalisations.csv')
evenements_train = pd.read_csv('../data_SUEZ/evenements_train.csv')
meteo_AlpesMaritimes = pd.read_csv('../data_SUEZ/meteo_AlpesMaritimes_pre_traite.csv')
nb_ev_train = pd.read_csv('../data_SUEZ/nb_ev_train.csv')
permut_train = pd.read_csv('../data_SUEZ/permut_train.csv')
stations_AlpesMaritimes = pd.read_csv('../data_SUEZ/stations_AlpesMaritimes.csv')

In [29]:
branchements.shape

(36179, 11)

In [30]:
evenements_train.shape

(2649, 6)

In [25]:
# Renaming columns

evenements_train.columns= ['GID_fuite', 'CODINSEE_fuite', 'date_fuite', 'GID_pose', 'COMPO_P', 'geometry_fuite']
branchements.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']
canalisations.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']

In [32]:
# Creating COMPO_P column in canalisation and branchements 

canalisations['COMPO_P'] = 'E_TRONCO'
branchements['COMPO_P'] = 'E_BRANCH'
elements = pd.concat([canalisations, branchements])

#evenements_train_br = evenements_train[ evenements_train['COMPO_P'] == 'E_BRANCH']
#evenements_train_canal = evenements_train[ evenements_train['COMPO_P'] == 'E_TRONCO']

In [63]:
# Merge elements on evenements (creation BDD complète)

elements_train = pd.merge(elements, evenements_train, on = 'GID_pose', how = 'left')

In [64]:
# Create BDD modèle 1ère fuite

elements_train['fuite'] = 
elements_train['GID_pose' elements_train['GID_fuite'].isnull().sum()

79076

In [27]:
# Put info from branchements in evenements_br 

merge_br = pd.merge(evenements_train_br, branchements, on = 'GID_pose')
sum(merge_br.CODINSEE_fuite != merge_br.CODINSEE_pose)

5

In [28]:
# Put info from canalisation in evenements_canal 

merge_canal = pd.merge(evenements_train_canal, canalisations, on = 'GID_pose')
sum(merge_canal.CODINSEE_fuite != merge_canal.CODINSEE_pose)

3